In [42]:
using LinearAlgebra, Statistics, StatsPlots, Random

### Simulation
Here, I extracted 1000 SNP loci of 100 random ID from a real data set.  Then I
1. sample 100 loci as QTL
2. sampel the QTL effect from a normal distribution
3. Calculate a true breeding values and G matrix

In [43]:
nid, nlc, nql = 100, 1000, 100

# read the SNP genotypes (W) as floats
W = convert.(Float64, reshape(read("gt.txt"), nlc, :)) .- 48

# sample QTL
qlc = sort(shuffle(1:nlc)[1:nql])

# and their effects
eql = randn(nql) ./ sqrt(nql)

# give each ID a random name
ID = String[]
for _ in 1:nid
    push!(ID, randstring(7))
end

# the QTL
qtl = W[qlc, :]

# the true breeding values
tbv = qtl'eql

# frequencies of the SNP loci
p = mean(W, dims=2)./2
q = 1 .- p

# ∑(2𝑝𝑞)
s2pq = 2p'q
r2pq = 1/s2pq[1]  # mul is faster than div

# the G matrix
W .-= 2p
G = W'W .* r2pq + 0.0001I;

From above, we obtained `ID` names, `true breeding values` and `G matrix`.

Next we:

### Create phenotypes with homogeneous residuals

In [44]:
h2 = 0.8
vg = var(tbv)
ve = vg/h2 * (1-h2)
se = sqrt(ve)

# Phenotype part I
y1 = tbv + rand(nid) .* se;

using the first 80 ID as a training set, the rest as validation set.

### EBV with MME

In [45]:
nt, nv = 80, 20

# design matrix for μ, assuming no other fixed effects
X = ones(nt)

# design matrix for ID
Z = [I zeros(nt, nv)]

#σ^2_a/σ^2_e
λ = (1-h2)/h2

# left hand side
giv = inv(G)
lhs = [ nid X'Z
        Z'X (Z'Z + giv .* λ)]

# right had side
y = y1[1:nt]
rhs = [ X'y
        Z'y]

# EBV
ebv = lhs \ rhs

# correlation in the training and validation sets
cor(tbv[1:nt], ebv[2:nt+1]), cor(tbv[nt+1:end], ebv[nt+2:end])

(0.9879358045967253, -0.02165307311304022)

### Create phenotypes with heterogeneous residuals

In [46]:
# suppose each training ID has 1-20 observations
nob = [rand(1:20, nt); ones(nv)]

# the residual 
err = randn(nid) .* se ./ sqrt.(nob)

# phenotype part II
y2 = tbv + err;

### EBV with MME

In [47]:
# inverse of residuals
R = diagm(nob[1:nt] ./ ve)

lhs = [ X'R*X    X'R*Z
        Z'R*X  Z'R*Z + giv./vg ]
rhs = [ X'R*y
        Z'R*y ]

ebv = lhs \ rhs

cor(tbv[1:nt], ebv[2:nt+1]), cor(tbv[nt+1:end], ebv[nt+2:end])

(0.9889878526582196, 0.049813754801272245)

### Other methods?
$y_v = G^{22}\cdot G^{12}\cdot G^{11}\cdot y_t$